In [ ]:
#Bow  Bag of Words  CountVectorizer
# 문서를 고정된 길이의 벡터로 변환
# 문서 - 단어행렬
# 장점 : 간단하고 빠름,
# 단점 : 단어순서 손실, 희소성, 의미적 유사성 무시

# tf-idf TfidfVectorizer
# 모든 문서에서 자주등장하는 단어의 영향을 줄이고 문서 특이 단어를 강조

# multinomal Naive Bayes  확률 모델
# LogisticRegression : 다중클래스  회귀기반 분류

# RidgeClassifer : 회귀 기반 분류 L2규제

# N-gram : 단점 차원폭발에 주의 (정규화/차원 축소 고려)

# Kolnpy Okt


In [3]:
import numpy as np
import pandas as pd
# scikit-learn
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.model_selection import train_test_split
#분류모델
from sklearn.naive_bayes import MultinomialNB
from sklearn.linear_model import LogisticRegression,RidgeClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier

In [ ]:
categories =  ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']
# # data load
# newsgroups_train =  fetch_20newsgroups(subset='train'
#                     ,remove = ('headers','footers','quotes')
#                     ,categories=categories
#                    )
# newsgroups_test =  fetch_20newsgroups(subset='test'
#                     ,remove = ('headers','footers','quotes')
#                     ,categories=categories
#                    )

categories =  ['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

- alt.atheism	무신론, 종교 비판 토론	종교의 존재, 신의 존재 유무, 종교적 주장 반박, 철학적 논쟁 등
- talk.religion.misc	일반 종교 토론 (기타 잡담 포함)	기독교, 불교, 이슬람 등 다양한 종교 관련 이야기, 개인 경험, 신념 공유 등
- comp.graphics	컴퓨터 그래픽스, 이미지 처리	3D 렌더링, 이미지 파일 포맷, 그래픽 소프트웨어 사용법, OpenGL 등 기술 관련 토론
- sci.space 우주 과학, 천문학 로켓, NASA, 행성 탐사, 외계 생명 가능성, 우주 물리학 관련 토론

In [50]:
from sklearn.datasets import load_files
train_path = r'C:\LLM\ch05\20newsbydate\20news-bydate-train'
test_path = r'C:\LLM\ch05\20newsbydate\20news-bydate-test'
newsgroups_train = load_files(train_path,encoding='latin1')
newsgroups_test = load_files(test_path,encoding='latin1')

In [51]:
categories

['alt.atheism', 'talk.religion.misc', 'comp.graphics', 'sci.space']

In [52]:
# 카테고리 제거
def filter_categories(dataset, categories):
    target_names = dataset.target_names
    selected_idx = [ target_names.index(c) for c in categories  ]
    #필터링
    data_filtered, target_filtered = [], []
    for text,label in zip(dataset.data, dataset.target):
        if label in selected_idx:
            new_label = selected_idx.index(label)  # 라벨 재 정렬
            data_filtered.append(text) ; target_filtered.append( new_label  )
    return data_filtered,target_filtered,categories

In [53]:
train_data, train_target, target_names = filter_categories(newsgroups_train,categories)
test_data, test_target, _ = filter_categories(newsgroups_test,categories)

In [41]:
# 해더 푸터 인용문 제거

In [54]:
import re

def clean_text(text):
    # 헤더 제거
    text = re.sub(r'^From:.*\n', '', text, flags=re.MULTILINE)
    text = re.sub(r'^Subject:.*\n', '', text, flags=re.MULTILINE)

    # 풋터 제거
    text = re.sub(r'\n--\n.*$', '', text, flags=re.DOTALL)

    # 인용문 제거
    text = re.sub(r'(^|\n)[>|:].*', '', text)

    return text

In [55]:
train_data = [ clean_text(t) for t in train_data]
test_data = [ clean_text(t) for t in test_data]

In [56]:
len(train_data), len(train_target), len(test_data), len(test_target)

(2034, 2034, 1353, 1353)

- 멀티노멀 나이즈베이즈
- 문서에 포함된 단어들의 출현 횟수를 기반으로 해서 그 문서가 어떤 주제에 속할지 확률적
- 스팸필터링, 뉴스기사 카테고리, 감성분석

- 베이즈정리 확률 이론 - 조건부 확률
- 단어A가 나왔을때 이 문서가 스팸 B 일 확률은 얼마

$P(\text{스팸} | \text{단어들}) = \frac{P(\text{단어들} | \text{스팸}) \cdot P(\text{스팸})}{P(\text{단어들})}$

- 나이브 Naive  : 순진한 가정
    - 가정 : 문서안의 모든 단어는 서로 독립적
    - 현실 : 스펨에 자주 나오는 단어들은 서로 독립적이지 않다
    - 실제 : 이러한 가정은 계산량을 빠르게 하고 단순하지만 정확도가 어느정도 나온다
- 멀티노멀 : 다항 분포
    - 의미 : 단어의 출현 횟수를 중요
    - 횟수를 세는 멀티노미얼 방식이 NLP 잘 맞는다
    - 모델은 단어의 빈도수 통계
    - 스펨메일 통계 spem
        - free : 150
        - money : 100
        - viagra : 50
        - report : 5
    - 정상메일 Ham 통계
        - report:80
        - metting : 60
        - free : 10
    - 이러한 통계를 바탕으로 이 카테고리에서 특정 단어가 나올 확률 P('free'|스펨) 을 모두 계산
    - "Free money meeting"
        - 스펨??
            - 기본스펨확률 x 스팸일때 free가 나올 확률 x 스팸일때 money 나올 확률 x 스팸일때 meeting 나올 확률
        - 정상
            - 기본정상확률 x 정상일때...   

In [60]:
# nltk tokenizer stemer
import nltk
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\playdata2\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [61]:
from nltk.corpus import stopwords
from nltk.tokenize import RegexpTokenizer
from nltk.stem.porter import PorterStemmer

In [62]:
# min_df : 단어의 빈도가 최소 5개의 문서에 등장  - 노이즈 감소
# max_df : 50% 너무흔한 단어는 제거
cv = CountVectorizer(max_features=2000,min_df=5, max_df=0.5)
x_train_cv = cv.fit_transform(train_data)
x_test_cv = cv.transform(test_data)
x_train_cv.shape,  x_test_cv.shape

((2034, 2000), (1353, 2000))

In [63]:
cv.get_feature_names_out()

array(['00', '000', '01', ..., 'zip', 'zoo', 'zoology'],
      shape=(2000,), dtype=object)

In [70]:
(x_train_cv[0].toarray()[0])[x_train_cv[0].toarray()[0]>0]

array([1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1,
       1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 2, 2, 1, 1, 1, 1])

In [78]:
# BOW 기반 + MNB
# 텍스트 분류의 강력한 baseline  희소데이터에 강함
# 모델선택
nb = MultinomialNB()
# 학습용데이터 벡터데이터
nb.fit(x_train_cv,train_target)
print(nb.score(x_train_cv, train_target)  , nb.score(x_test_cv,test_target))
# 분류 리포트
from sklearn.metrics import classification_report
y_pred_nb = nb.predict(x_test_cv)
print( classification_report(test_target,y_pred_nb,target_names=categories)  )

0.9110127826941986 0.8033998521803399
                    precision    recall  f1-score   support

       alt.atheism       0.75      0.75      0.75       319
talk.religion.misc       0.68      0.63      0.65       251
     comp.graphics       0.85      0.92      0.88       389
         sci.space       0.88      0.84      0.86       394

          accuracy                           0.80      1353
         macro avg       0.79      0.79      0.79      1353
      weighted avg       0.80      0.80      0.80      1353



In [77]:
# TF-IDF + MNB + LogisticRegression
# TF-IDF로 중요단어 강조, 선형모델과 자주사용  BOW 대비 흔한 단어 영향 감소
tfidf = TfidfVectorizer(max_features=2000, min_df=5, max_df=0.5)
x_train_tfid = tfidf.fit_transform(train_data)
x_test_tfid = tfidf.transform(test_data)

# NB + tf-idf 
nb_tfidf  = MultinomialNB()
nb_tfidf.fit(x_train_tfid,train_target)
print(nb_tfidf.score(x_train_tfid,train_target),  nb_tfidf.score(x_test_tfid, test_target) )

0.9311701081612586 0.8159645232815964


In [84]:
# logistic 
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test \
    = train_test_split(x_train_tfid,train_target,test_size=0.2
                       ,stratify=train_target,random_state=42)
lr = LogisticRegression(max_iter=1000)
lr.fit(x_train, y_train)
print(lr.score(x_train,y_train),  lr.score(x_test, y_test) )

0.9698832206515058 0.8894348894348895


In [94]:
# 과적합 해결을 위한 규제
rc = RidgeClassifier(alpha=10)
rc.fit(x_train, y_train)
print(rc.score(x_train,y_train),  rc.score(x_test, y_test) )

0.9330055316533498 0.8722358722358723


In [97]:
# L1 규제   L1 Logistic(Lasso와 유사)
# 일부 계수를 0으로 만들어서 특성 선택을 수행.. 중요피처 select 효과
l1_lr = LogisticRegression(penalty='l1',max_iter=1000,solver='saga')
l1_lr.fit(x_train,y_train)
print(l1_lr.score(x_train,y_train),  l1_lr.score(x_test, y_test) )

0.8905961893054702 0.8353808353808354


In [ ]:
# 트리모델 + tfidf
tree = DecisionTreeClassifier()
fores = RandomForestClassifier()
gb = GradientBoostingClassifier()

array([[0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.07046516, 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]], shape=(1627, 2000))